# Pipeline for YOLOv8 + DeepSORT tracking

In [3]:
%cd Multiple-Object-Tracking

/Users/macjanine/Documents/Studium/Master/3.Semester/small-toys-object-tracking/YOLOv8_Deepsort/Multiple-Object-Tracking


In [7]:
import cv2
import ultralytics
ultralytics.checks()

Ultralytics 8.4.5 🚀 Python-3.11.14 torch-2.9.1 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 226.3/228.3 GB disk)


In [ ]:
# Model path
yolo_model_path = 'best6.pt'
siamese_model_path = 'model640.pt'

# Video input and output paths
video_path = 'output_10fps.mp4'
video_out_path = 'pred_10fps.mp4'

# Detections file
filename = 'detections_10fps.txt'
tracking_file = 'tracking_10fps.txt'

In [53]:
# YOLOv8 detection
yolo = YOLO(yolo_model_path)

# Open video file
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

# Initialize video writer
cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

detection_threshold = 0.5
frame_number = -1

# Open a text file to save detections
with open(filename, 'w') as f:
    while ret:
        frame_number += 1

        # Perform object detection using YOLOv8
        results = yolo.predict(frame)

        # Collect detections
        detections = []
        for result in results:
            for r in result.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = r
                x1, x2 = int(x1), int(x2)
                y1, y2 = int(y1), int(y2)
                class_id = int(class_id)
                if score > detection_threshold:
                    width = x2 - x1
                    height = y2 - y1
                    center_x = x1 + width // 2
                    center_y = y1 + height // 2
                    detections.append([center_x, center_y, width, height, score])
                    # Save detection to text file
                    f.write(f'{frame_number},-1,{center_x},{center_y},{width},{height},{score},-1,-1,-1\n')
        # Write the frame to the output video
        cap_out.write(frame)

        # Read the next frame
        ret, frame = cap.read()


0: 640x384 4 hexbugs, 93.0ms
Speed: 2.3ms preprocess, 93.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)



OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


0: 640x384 4 hexbugs, 81.2ms
Speed: 1.6ms preprocess, 81.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 hexbugs, 82.5ms
Speed: 1.4ms preprocess, 82.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 hexbugs, 78.5ms
Speed: 2.1ms preprocess, 78.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 hexbugs, 93.5ms
Speed: 1.5ms preprocess, 93.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 hexbugs, 85.0ms
Speed: 1.9ms preprocess, 85.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 hexbugs, 89.3ms
Speed: 2.2ms preprocess, 89.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 hexbugs, 76.9ms
Speed: 1.2ms preprocess, 76.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 hexbugs, 84.1ms
Speed: 1.7ms preprocess, 84.1ms inference, 0.4ms postprocess per image at shape (1

In [54]:
cd object_tracking

/Users/macjanine/Documents/Studium/Master/3.Semester/small-toys-object-tracking/YOLOv8_Deepsort/Multiple-Object-Tracking/object_tracking


In [56]:
from test_on_video import get_dict, get_gt, deepsort_rbc
import numpy as np

# Load detections for the video. Options available: yolo, ssd, and mask-rcnn
gt_dict = get_dict("../"+filename)
print(gt_dict)

tmp_cap = cv2.VideoCapture("../"+video_path)
tmp_shape = tmp_cap.read()[1].shape

cap = cv2.VideoCapture("../"+video_path)

# An optional mask for the given video, to focus on the road.
# mask = get_mask('roi.jpg')

# Initialize deep sort.
deepsort = deepsort_rbc(wt_path="../"+siamese_model_path)

frame_id = 1

# mask = np.expand_dims(mask, 2)
# mask = np.repeat(mask, 3, 2)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("../"+video_out_path, fourcc, 15.0, (tmp_shape[1], tmp_shape[0]))

# Open a text file to write ID and bounding boxes
with open("../"+tracking_file, 'w') as f:
    while True:
        print(frame_id)
        ret, frame = cap.read()
        if not ret:
            frame_id += 1
            break

        # frame = frame * mask
        frame = frame.astype(np.uint8)

        # Here it gets info for each frame
        try:
            detections, out_scores = get_gt(frame, frame_id, gt_dict)
            print(detections)
        except:
            frame_id += 1
            continue

        if detections is None:
            print("No dets")
            frame_id += 1
            continue

        detections = np.array(detections)
        print(detections)
        out_scores = np.array(out_scores)
        print(out_scores)

        tracker, detections_class = deepsort.run_deep_sort(frame, out_scores, detections)

        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue

            bbox = track.to_tlbr()  # Get the corrected/predicted bounding box
            id_num = str(track.track_id)  # Get the ID for the particular track
            features = track.features  # Get the feature vector corresponding to the detection

            # Draw bbox from tracker
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 255, 255), 2)
            cv2.putText(frame, str(id_num), (int(bbox[0]), int(bbox[1])), 0, 5e-3 * 200, (0, 255, 0), 2)

            # Write ID and bounding box to the file
            f.write(f'{frame_id},{id_num},{int(bbox[0])},{int(bbox[1])},{int(bbox[2])},{int(bbox[3])}\n')

            # Draw bbox from detector. Just to compare.
            for det in detections_class:
                bbox = det.to_tlbr()
                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 255, 0), 2)

        # cv2.imshow('frame', frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += 1


{0: [{'coords': [285.0, 1270.0, 52.0, 53.0], 'conf': 0.7700116634368896}, {'coords': [688.0, 669.0, 50.0, 48.0], 'conf': 0.710455596446991}, {'coords': [162.0, 685.0, 49.0, 50.0], 'conf': 0.6211811304092407}, {'coords': [278.0, 680.0, 45.0, 42.0], 'conf': 0.539483904838562}], 1: [{'coords': [656.0, 1736.0, 55.0, 53.0], 'conf': 0.684565544128418}, {'coords': [681.0, 795.0, 50.0, 50.0], 'conf': 0.6642363667488098}, {'coords': [175.0, 658.0, 44.0, 44.0], 'conf': 0.561223566532135}], 2: [{'coords': [709.0, 1729.0, 55.0, 55.0], 'conf': 0.6755041480064392}, {'coords': [658.0, 842.0, 49.0, 49.0], 'conf': 0.6286309361457825}, {'coords': [250.0, 670.0, 47.0, 45.0], 'conf': 0.6119263172149658}, {'coords': [164.0, 658.0, 44.0, 43.0], 'conf': 0.5406264662742615}], 3: [{'coords': [121.0, 658.0, 48.0, 47.0], 'conf': 0.6995590925216675}, {'coords': [644.0, 912.0, 45.0, 47.0], 'conf': 0.646828830242157}, {'coords': [250.0, 1550.0, 49.0, 48.0], 'conf': 0.5664522051811218}, {'coords': [256.0, 670.0, 46.